In [1]:
import ClimaAtmos as CA
import ClimaUtilities
import ClimaUtilities: TimeVaryingInputs
using Dates
import ClimaCoreTempestRemap
import ClimaCore.Fields

┌ Warning: Circular dependency detected. Precompilation will be skipped for:
│   GridLayoutBase [3955a311-db13-416c-9275-1d80ed98e5e9]
│   PreallocationTools [d236fae5-4411-538c-8e31-a6e3d9e00b46]
│   SpecialFunctionsExt [1285c0f1-ff9f-5867-b66e-0f359bcf09ba]
│   SciMLOperatorsStaticArraysCoreExt [a2df0a61-553a-563b-aed7-0ce21874eb58]
│   DiffEqBaseChainRulesCoreExt [b00db79b-61e3-50fb-b26f-2d35b2d9e4ed]
│   StaticArrayInterface [0d7ed370-da01-4f52-bd93-41d350b8b718]
│   RecursiveArrayToolsFastBroadcastExt [42296aa8-c874-5f57-b5c1-8d6f5ebd5400]
│   SciMLBase [0bca4576-84f4-4d90-8ffe-ffa030f20462]
│   GPUArrays [0c68f7d7-f131-5f86-a1c3-88cf8149b2d7]
│   DiffEqBaseCUDAExt [78064ecb-8a7e-51fd-a998-f0197ddca326]
│   KernelAbstractions [63c18a36-062a-441e-b654-da1e3ab1ce7c]
│   CreateParametersExt [de10cf08-5a56-53da-a061-ea54c6b27d66]
│   RecursiveArrayToolsStructArraysExt [07510ace-05c5-59ac-8535-cc0df8d4a157]
│   RecursiveArrayToolsSparseArraysExt [73e54eaf-3344-511d-b088-1ac5413eca63]
│

In [2]:
pwd()

"/home/jschmitt/ClimaAtmos.jl/examples/tvforcing"

In [11]:
cd("../../")

In [12]:
# get simulation object so as to get target_space
config = CA.AtmosConfig("config/model_configs/prognostic_edmfx_gcmdriven_column.yml")
simulation = CA.get_simulation(config)

[ Info: Loading yaml file config/default_configs/default_config.yml
[ Info: Loading yaml file config/model_configs/prognostic_edmfx_gcmdriven_column.yml
[ Info: Running on CPUSingleThreaded
[ Info: Setting up single-process ClimaAtmos run
[ Info: Running ClimaCore in unthreaded mode
┌ Info: Making AtmosConfig with config files: 
│    config/default_configs/default_config.yml
└    config/model_configs/prognostic_edmfx_gcmdriven_column.yml
[ Info: Using CAM_SE hyperdiffusion. vorticity_hyperdiffusion_coefficient, scalar_hyperdiffusion_coefficient and divergence_damping_factor in the config will be ignored.
┌ Info: numerics 
│            `energy_upwinding`::`Val{:vanleer_limiter}`
│            `tracer_upwinding`::`Val{:vanleer_limiter}`
│             `edmfx_upwinding`::`Val{:first_order}`
│     `edmfx_sgsflux_upwinding`::`Val{:none}`
│     `test_dycore_consistency`::`Nothing`
└                     `limiter`::`Nothing`
┌ Info: AtmosModel: 
│ 
│                       `moisture_model`::`Clim

Simulation prognostic_edmfx_gcmdriven_column
├── Running on: CPUSingleThreaded
├── Output folder: output/prognostic_edmfx_gcmdriven_column/output_0014
├── Start date: 2010-01-01T00:00:00
├── Current time: 0.0 seconds
└── Stop time: 21600.0 seconds

In [7]:
typeof(simulation.integrator.u.c.ρ)

ClimaCore.Fields.Field{ClimaCore.DataLayouts.VIJFH{Float32, 60, 1, SubArray{Float32, 5, Array{Float32, 5}, Tuple{Base.Slice{Base.OneTo{Int64}}, Base.Slice{Base.OneTo{Int64}}, Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}, Base.Slice{Base.OneTo{Int64}}}, false}}, ClimaCore.Spaces.ExtrudedFiniteDifferenceSpace{ClimaCore.Grids.ExtrudedFiniteDifferenceGrid{ClimaCore.Grids.SpectralElementGrid2D{ClimaCore.Topologies.Topology2D{ClimaComms.SingletonCommsContext{ClimaComms.CPUSingleThreaded}, ClimaCore.Meshes.RectilinearMesh{ClimaCore.Meshes.IntervalMesh{ClimaCore.Domains.IntervalDomain{ClimaCore.Geometry.XPoint{Float32}, Nothing}, LinRange{ClimaCore.Geometry.XPoint{Float32}, Int64}}, ClimaCore.Meshes.IntervalMesh{ClimaCore.Domains.IntervalDomain{ClimaCore.Geometry.YPoint{Float32}, Nothing}, LinRange{ClimaCore.Geometry.YPoint{Float32}, Int64}}}, Vector{CartesianIndex{2}}, Matrix{Int64}, Vector{Tuple{Int64, Int64, Int64, Int64, Bool}}, Vector{Tuple{Int64, Int64, Int64, Int64, Bool}}, Vector{Tu

In [5]:
Fields.coordinate_field(simulation.integrator.u.c.ρ)

ClimaCore.Geometry.XYZPoint{Float32}-valued Field:
  x: Float32[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
  y: Float32[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
  z: Float32[15.0, 46.4423, 80.9064, 118.68, 160.076, 205.44, 255.145, 309.601, 369.254, 434.591  …  23340.1, 24915.8, 26546.9, 28229.2, 29957.5, 31726.0, 33528.5, 35357.8, 37206.8, 39067.6]

In [6]:
using NCDatasets
ds = NCDataset("/home/jschmitt/ClimaAtmos.jl/examples/tvforcing/sim_forcing_site23_ng_xyz.nc")

# ds

using ClimaUtilities.FileReaders
# test file read
test = FileReaders.NCFileReader("/home/jschmitt/ClimaAtmos.jl/examples/tvforcing/sim_forcing_site23_ng_xyz.nc", "ta")

ClimaUtilitiesNCDatasetsExt.NCFileReaderExt.NCFileReader{Vector{String}, String, Tuple{Vector{Float64}, Vector{Float64}, Vector{Float64}}, NCDataset{Nothing, Missing}, Vector{DateTime}, typeof(identity), ClimaUtilities.DataStructures.LRUCache{DateTime, Array}}(["/home/jschmitt/ClimaAtmos.jl/examples/tvforcing/sim_forcing_site23_ng_xyz.nc"], "ta", ([0.0, 1.0], [0.0, 1.0], [136.7820652123418, 357.10261550680383, 581.6044427454976, 810.8070676844417, 1045.110618184304, 1284.7746406993285, 1530.7430071090305, 1784.936508505352, 2046.6933255287963, 2315.3707552879714  …  18668.63771024466, 20691.374288566094, 23863.194656812768, 26472.848284063883, 31088.91564729868, 33521.62552030241, 35854.983382178034, 39510.746103041114, 42539.11490188583, 47856.81012147466]), ("x", "y", "z"), [DateTime("2007-07-01T00:00:00"), DateTime("2007-07-01T01:00:00"), DateTime("2007-07-01T02:00:00"), DateTime("2007-07-01T03:00:00"), DateTime("2007-07-01T04:00:00"), DateTime("2007-07-01T05:00:00"), DateTime("2007

In [11]:
ds["x"][:]

2-element Vector{Float64}:
 0.0
 1.0

In [12]:
ClimaUtilities.DataHandling.DataHandler(
    "/home/jschmitt/ClimaAtmos.jl/examples/tvforcing/sim_forcing_site23_ng_xyz.nc",
    "ta",
    axes(similar(simulation.integrator.u.c.ρ)),
);

In [104]:
dim_names = NCDatasets.dimnames(ds["ta"])
is_time = x -> x == "time" || x == "date" || x == "t"
dim_names = filter(!is_time, dim_names)


all(d in keys(ds) for d in dim_names)

true

In [13]:
start_time = DateTime(2007, 7, 1)
tv = TimeVaryingInputs.TimeVaryingInput(["/home/jschmitt/ClimaAtmos.jl/examples/tvforcing/sim_forcing_site23_ng_xyz.nc"],
                                ["ta"],
                                axes(similar(simulation.integrator.u.c.ρ));
                                reference_date = start_time,
                                regridder_type = :InterpolationsRegridder,
);

In [26]:
import ClimaCore:Fields
F1 = zero(simulation.integrator.u.c.ρ)
next_t = start_time + Dates.Second(100000)
next_t
TimeVaryingInputs.evaluate!(F1, tv, next_t)

In [38]:
axes(similar(Fields.level(simulation.integrator.u.c.ρ, 1), FT))

LoadError: Struct type Float64 cannot be represented using base type Float32

In [1]:
import ClimaCore: Fields
FT = Float32
tv = TimeVaryingInputs.TimeVaryingInput(["/home/jschmitt/ClimaAtmos.jl/examples/tvforcing/sim_forcing_site23_ng_xyz.nc"],
                                ["coszen"],
                                axes(similar(Fields.level(simulation.integrator.u.c.ρ, 1), FT));
                                reference_date = start_time,
                                regridder_type = :InterpolationsRegridder,
);
# F1 = zero(simulation.integrator.u.c.ρ)
# next_t = start_time + Dates.Second(100000)
# next_t
# TimeVaryingInputs.evaluate!(F1, tv, next_t)


┌ Warning: Circular dependency detected. Precompilation will be skipped for:
│   GridLayoutBase [3955a311-db13-416c-9275-1d80ed98e5e9]
│   PreallocationTools [d236fae5-4411-538c-8e31-a6e3d9e00b46]
│   SpecialFunctionsExt [1285c0f1-ff9f-5867-b66e-0f359bcf09ba]
│   SciMLOperatorsStaticArraysCoreExt [a2df0a61-553a-563b-aed7-0ce21874eb58]
│   DiffEqBaseChainRulesCoreExt [b00db79b-61e3-50fb-b26f-2d35b2d9e4ed]
│   StaticArrayInterface [0d7ed370-da01-4f52-bd93-41d350b8b718]
│   RecursiveArrayToolsFastBroadcastExt [42296aa8-c874-5f57-b5c1-8d6f5ebd5400]
│   SciMLBase [0bca4576-84f4-4d90-8ffe-ffa030f20462]
│   GPUArrays [0c68f7d7-f131-5f86-a1c3-88cf8149b2d7]
│   DiffEqBaseCUDAExt [78064ecb-8a7e-51fd-a998-f0197ddca326]
│   KernelAbstractions [63c18a36-062a-441e-b654-da1e3ab1ce7c]
│   CreateParametersExt [de10cf08-5a56-53da-a061-ea54c6b27d66]
│   RecursiveArrayToolsStructArraysExt [07510ace-05c5-59ac-8535-cc0df8d4a157]
│   RecursiveArrayToolsSparseArraysExt [73e54eaf-3344-511d-b088-1ac5413eca63]
│

LoadError: UndefVarError: `simulation` not defined